In [1]:
# INFO: This notebook is for reference purposes only, it describes how ADM.duckdb was populated and may not work in the future (read only DB)

import pandas as pd
import geopandas as gpd

from satellite.geo.functional import session
from satellite.geo.models import ADM0, ADM1, ADM2

### ARG 🇦🇷

#### ADM1

In [2]:
df_adm1_ARG = gpd.read_file("provincias.geojson") # https://github.com/PoliticaArgentina/data_warehouse/blob/master/geoAr/data_raw/provincias.geojson
df_adm1_ARG = df_adm1_ARG[["codprov_censo", "provincia"]]
df_adm1_ARG = df_adm1_ARG.rename(columns={"codprov_censo": "adm1", "provincia": "name"})
df_adm1_ARG["adm0"] = "ARG"
df_adm1_ARG.head()

,adm1,name,adm0
0,02,Ciudad Autónoma de Buenos Aires,ARG
1,10,Catamarca,ARG
2,14,Córdoba,ARG
3,18,Corrientes,ARG
4,22,Chaco,ARG


In [3]:
with session() as sess:
    for adm in [ADM0, ADM1]:
        adm.drop_table()
        adm.create_table()
        
    sess.execute("INSERT INTO adm0 (code, name) VALUES ('ARG', 'Argentina')")
    sess.execute("INSERT INTO adm0 (code, name) VALUES ('BRA', 'Brazil')")
    sess.execute("INSERT INTO adm1 SELECT * FROM df_adm1_ARG")
    sess.commit()

In [4]:
ADM0.get(code="ARG")

Argentina

In [5]:
ADM1.filter(adm0="ARG")[:5]

[Ciudad Autónoma de Buenos Aires, Catamarca, Córdoba, Corrientes, Chaco]

In [6]:
ADM1.filter(name="Catamarca", adm0="ARG")

[Catamarca]

In [7]:
ADM1.filter(code="10", adm0="ARG")

[Catamarca]

#### ADM2

In [8]:
df_adm2_ARG = gpd.read_file("censo_2010.geojson") # https://github.com/PoliticaArgentina/data_warehouse/blob/master/geoAr/data_raw/censos/censo_2010.geojson
df_adm2_ARG = df_adm2_ARG[["prov_cod", "link", "depto", "geometry"]]
df_adm2_ARG = df_adm2_ARG.rename(columns={"prov_cod": "adm1", "link": "adm2", "depto": "name"})
"""
df_gpkg = df_adm2_ARG[["adm1", "adm2", "geometry"]]
df_gpkg.to_file("ARG.gpkg", driver="GPKG")
"""
df_adm2_ARG.head()

,adm1,adm2,name,geometry
0,02,02001,Comuna 01,"POLYGON ((-58.36801 -34.57821, -58.36798 -34.5..."
1,02,02002,Comuna 02,"POLYGON ((-58.3944 -34.59968, -58.39471 -34.59..."
2,02,02003,Comuna 03,"POLYGON ((-58.4103 -34.59809, -58.40943 -34.59..."
3,02,02004,Comuna 04,"MULTIPOLYGON (((-58.35987 -34.62558, -58.35978..."
4,02,02005,Comuna 05,"POLYGON ((-58.41566 -34.59786, -58.41463 -34.5..."


In [9]:
df_adm2_ARG["adm0"] = "ARG"
df_adm2_ARG = df_adm2_ARG[["adm2", "name", "adm0", "adm1"]]
df_adm2_ARG.head()

,adm2,name,adm0,adm1
0,02001,Comuna 01,ARG,02
1,02002,Comuna 02,ARG,02
2,02003,Comuna 03,ARG,02
3,02004,Comuna 04,ARG,02
4,02005,Comuna 05,ARG,02


In [10]:
with session() as sess:
    ADM2.drop_table()
    ADM2.create_table()

    sess.execute("INSERT INTO adm2 SELECT * FROM df_adm2_ARG")
    sess.commit()

In [11]:
ADM2.filter(adm0="ARG")[:5]

[Comuna 01, Comuna 02, Comuna 03, Comuna 04, Comuna 05]

In [12]:
ADM2.filter(adm1="02", adm0="ARG")

[Comuna 01,
 Comuna 02,
 Comuna 03,
 Comuna 04,
 Comuna 05,
 Comuna 06,
 Comuna 07,
 Comuna 08,
 Comuna 09,
 Comuna 10,
 Comuna 11,
 Comuna 12,
 Comuna 13,
 Comuna 14,
 Comuna 15]

In [13]:
ADM2.get(code="02001", adm0="ARG")

Comuna 01

In [14]:
ADM2.get(name="Comuna 01", adm0="ARG")

Comuna 01

### BRA 🇧🇷

https://www.ibge.gov.br/geociencias/organizacao-do-territorio/estrutura-territorial/15774-malhas.html

In [15]:
df_adm1_BRA = gpd.GeoDataFrame.from_file("BR_UF_2022.dbf")
df_adm1_BRA = df_adm1_BRA[["CD_UF", "NM_UF"]]
df_adm1_BRA = df_adm1_BRA.rename(columns={"CD_UF": "adm1","NM_UF": "name"})
df_adm1_BRA["adm0"] = "BRA"
df_adm1_BRA.head()

,adm1,name,adm0
0,12,Acre,BRA
1,13,Amazonas,BRA
2,15,ParÃ¡,BRA
3,16,AmapÃ¡,BRA
4,17,Tocantins,BRA


In [16]:
with session() as sess:
    sess.execute("DELETE FROM adm1 WHERE adm0 = 'BRA'")
    sess.execute("INSERT INTO adm1 SELECT * FROM df_adm1_BRA")
    sess.commit()

In [17]:
ADM0.get(code="BRA")

Brazil

In [18]:
ADM1.filter(adm0="BRA")[:5]

[Acre, Amazonas, ParÃ¡, AmapÃ¡, Tocantins]

In [19]:
ADM1.filter(name="Santa Catarina", adm0="BRA")

[Santa Catarina]

In [20]:
ADM1.filter(code="42", adm0="BRA")

[Santa Catarina]

In [21]:
df_adm2_BRA = gpd.GeoDataFrame.from_file("BR_Municipios_2022.dbf")
df_adm2_BRA["adm1"] = df_adm2_BRA['CD_MUN'].astype(str).str[:2]
df_adm2_BRA = df_adm2_BRA.rename(columns={"CD_MUN": "adm2", "NM_MUN": "name"})
"""
df_gpkg = df_adm2_BRA[["adm1", "adm2", "geometry"]]
df_gpkg.to_file("BRA.gpkg", driver="GPKG")
"""
df_adm2_BRA["adm0"] = "BRA"
df_adm2_BRA = df_adm2_BRA[["adm2", "name", "adm0", "adm1"]]
df_adm2_BRA.head()

,adm2,name,adm0,adm1
0,1100015,Alta Floresta D'Oeste,BRA,11
1,1100023,Ariquemes,BRA,11
2,1100031,Cabixi,BRA,11
3,1100049,Cacoal,BRA,11
4,1100056,Cerejeiras,BRA,11


In [22]:
with session() as sess:
    sess.execute("DELETE FROM adm2 WHERE adm0 = 'BRA'")
    sess.execute("INSERT INTO adm2 SELECT * FROM df_adm2_BRA")
    sess.commit()

In [23]:
ADM2.filter(adm0="BRA")[:5]

[Alta Floresta D'Oeste, Ariquemes, Cabixi, Cacoal, Cerejeiras]

In [24]:
ADM2.get(code="3304557", adm0="BRA")

Rio de Janeiro